<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:200px; margin: 10px;" />
</div>

# Generative text model

In this notebook, we will load and use a generative language model that can produce a continuation for a given text. Learn more about the Text Generation task <a href="https://huggingface.co/tasks/text-generation" target="_blank" rel="noopener">here</a>. Please also refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a> to find more information about the UDF used in this notebook.

To execute queries and load data from the Exasol database we will be using the <a href="https://github.com/exasol/pyexasol" target="_blank" rel="noopener">`pyexasol`</a> module.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI Lab](../main_config.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

## Setup

### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

## Get a language model

To demonstrate the text generation task we will use [Open Pretrained Transformers (OPT)](https://huggingface.co/facebook/opt-125m), a decoder-only pre-trained transformer from Facebook.

We need to load the model from the Hugging Face Hub into the [BucketFS](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm). This could potentially be a long process, depending on the connection of the Database. Unfortunately, we cannot tell exactly when it has finished. The notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for a few moments after that, before querying the model.

You might see a warning that some weights are newly initialized and the model should be trained on a down-stream task. Please ignore this warning. For the purpose of this demonstration, it is not important, the model should still be able to produce some meaningful output.

In [ ]:
from exasol.nb_connector.model_installation import install_model, TransformerModel
from transformers import AutoModelForCausalLM

# This is the name of the model at the Hugging Face Hub
MODEL_NAME = 'facebook/opt-125m'
install_model(ai_lab_config, TransformerModel(MODEL_NAME, 'text_generation', AutoModelForCausalLM))

## Use the language model

Let's put our starting text in a variable. We will feed this text to the `TE_TEXT_GENERATION_UDF`, which will use the previously downloaded model to generate a continuation of the input text.

In [ ]:
MY_TEXT = 'The bar-headed goose can fly at much'

# Make sure our texts can be used in an SQL statement.
MY_TEXT = MY_TEXT.replace("'", "''")

In [ ]:
from exasol.nb_connector.connections import open_pyexasol_connection
from exasol.nb_connector.language_container_activation import get_activation_sql

# We will be creating a new database session every time we call the model.
# We will have to activate the language container for each of these sessions.
# Here we will get the activation SQL command.
activation_sql = get_activation_sql(ai_lab_config)

The udf takes various input parameters:

* device_id: To run on a GPU, specify the valid cuda device ID.
* bucketfs_conn: The BucketFS connection name.
* sub_dir: The directory where the model is stored in the BucketFS.
* model_name: The name of the model to use for prediction.
* text_data: The text continue.
* max_length: The maximum total length of text to be generated.
* return_full_text: If set to FALSE, only added text is returned, otherwise the full text is returned.

You need to supply these parameters in the correct order. Further information can be found in the  <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a>.

We will use the `max_lenght` paramter to put a limit on the length of text the model can generate in one call.

In [ ]:
MAX_LENGTH = 30

At the start, the `MY_TEXT` variable gives an initial context to the generated text. We will update this variable with the generated text after each call to the udf, making it add to the text each time.
The generated text can be found in the `generated_text` column of the udf-output.

You can run the next cell multiple times to see how the text evolves with multiple calls.

In [ ]:
sql = f"""
SELECT {ai_lab_config.db_schema}.TE_TEXT_GENERATION_UDF(
    NULL,
    '{ai_lab_config.bfs_connection_name}',
    '{ai_lab_config.bfs_model_subdir}',
    '{MODEL_NAME}',
    '{MY_TEXT}',
    {MAX_LENGTH},
    True
)
"""

with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(query=activation_sql)
    result = conn.export_to_pandas(query_or_table=sql).squeeze()
    MY_TEXT = result['GENERATED_TEXT']
    # Errors can be observed at result['ERROR_MESSAGE']

print(MY_TEXT)
MY_TEXT = MY_TEXT.replace("'", "''")

 If you need more details to your output, you can find information on all output columns in the <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a>.